# MyMedScribe - A Performant and Reliable AI Medical Scribe

This Jupyter Notebook shows how to evaluate an LLM App unsing 
[Lynxius](https://www.lynxius.ai/) SDK.

The application being under examination is **MyMedScribe**, an AI Medical Scribe that 
assists medical doctors by capturing notes during patient visits. The App workflow 
follows:

1. 🎤 Notes are recorded with the the doctor's smartphone 🎙️
2. 🧠 **MyMedScribe**'s cleaver AI filters out small talk, summarizes content, identifies 
    key medical information and generates clinical documentation 🤖
3. 🩺 The doctor reviews the AI-generated notes and signs off ✅

## The Patient Consultation

* **Brenda Hills** experiences chest pain and seeks medical advice from her physician. 
* Her physician, **Dr. Sara Taylor**, utilizes the **MyMedScribe** App to streamline her 
  documentation process.

For Dr. Taylor, it is imperative that the transcripts and clinical notes produced by 
**MyMedScribe** are of exceptional quality: they need to be accurate, comprehensive, and 
free from any errors, ensuring no vital information is omitted.

<style>
    @media (prefers-color-scheme: dark) {
        .adaptive-text { background-color: #fffb00; } /* Light text in dark theme */
    }
    @media (prefers-color-scheme: light) {
        .adaptive-text { background-color: #fffb00; } /* Dark text in light theme */
    }
</style>

## Jupyter Notebook Setup

This Jupyter notebook requires a small setup to start. We need to import the relevant 
packages and utility functions. To consume [Lynxius Platform](https://platform.lynxius.ai/) 
we need to provide `LYNXIUS_API_KEY` secret key.
Today Lynxius offers OpenAI GPT-4 by default as LLM model for 
evaluation, but we are building our own models for more accurate evaluations.

⚠️⚠️⚠️ <span class="adaptive-text">At the moment Lynxius's OpenAI key is used, but we plan 
to enable also users to use their own keys or directly use our proprietary models. 
</span> ⚠️⚠️⚠️

In [1]:
import os
import sys
from getpass import getpass

from AI_medical_scribe_utils import load_input, load_yaml_input

# Making sure a local version of the Lynxius library is available
sys.path.append("../")


# set Lynxius secret key
if not (lynxius_api_key := os.getenv("LYNXIUS_API_KEY")):
    lynxius_api_key = getpass("🔑 Enter your Lynxius API key: ")

os.environ["LYNXIUS_API_KEY"] = lynxius_api_key

os.environ["LYNXIUS_BASE_URL"] = "http://REQUEST-US-WHICH-ENDOINT-TO-USE-TO-GET-ACCESS"

## Task-1: Evaluating Summarization Quality

**MyMedScribe** App filters and summarizes the patient visit and generates a clean 
transcript.

<img src="https://github-public-assets.s3.us-west-1.amazonaws.com/AI_medical_scribe_sumarization.png" alt="AI Medical Scribe summarization" width="60%" />

⚠️⚠️⚠️ **The AI-generated summarization looks good, but... How good is it?** ⚠️⚠️⚠️

**MyMedScribe** team knows how crucial it is to Dr. Taylor that the AI-generated 
transcript contains all the relevant information of the original conversation with 
Brenda. **MyMedScribe** team must evaluate their product before and after deploying it to 
production and chooses [Lynxius](https://www.lynxius.ai/) for testing and evaluating 
their LLM App.

### Missing Information Score

**MyMedScribe** team wants to evaluate the summarization performance of their LLM App. 
They care to know if the AI-generated transcript is semantically similar to the original 
raw transcript recorded by Dr. Taylor's smartphone and if it missing any relevant 
information.

[Lynxius](https://www.lynxius.ai/) `BertScore` evaluator is perfect as the 
**"Missing Information Score"**. It takes the original raw transcript and the 
AI-generated summary as inputs and determines their semanthic similarity.

<img src="https://github-public-assets.s3.us-west-1.amazonaws.com/AI_medical_scribe_eval_sumarization.png" alt="AI Medical Scribe eval summarization" width="60%" />

Let's see the `BertScore` evaluator in action:

In [2]:
# load raw transcript and AI-generated summary
raw_transcript = load_input('./data/AI_medical_scribe_raw.yaml')
ai_summary = load_input('./data/AI_medical_scribe_summary.yaml')

In [3]:
from lynxius import Lynxius

client = Lynxius()

# Calculate the BertScore and upload results to Lynxius Platform
client.evaluate(
    title="Bert Score v1",
    evaluator="bert_score",
    reference=raw_transcript,
    output=ai_summary,
    level="sentence",
    presence_threshold=0.65,
)

Success: {"uuid":"b34371a3-5074-433c-ac74-7615f0f9fa68"}


We previously set our Project's Bert Score Thresholds to the following values:

<table>
  <thead style="background-color: #919191;">
    <tr>
      <th>Metric</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Precision</td>
      <td>0.75</td>
    </tr>
    <tr>
      <td>Recall</td>
      <td>0.55</td>
    </tr>
    <tr>
      <td>F1</td>
      <td>0.60</td>
    </tr>
  </tbody>
</table>


Below a screenshot of the Bert Score results from [Lynxius Platform](platform.lynxius.ai).

<img src="https://github-public-assets.s3.us-west-1.amazonaws.com/AI_medical_scribe_BertScore_v1.png" alt="AI Medical Scribe BertScore v1" width="90%" />

❌❌❌ **Ouch!!! something went wrong** ❌❌❌

Looks like `BERTScore Precision` and `BERTScore Recall` are lower than the 
set threshold. Thankfully `BERTScore` also produce a list of tokens (sentences in our 
case) that are not present in the summary and [Lynxius Platform](platform.lynxius.ai) 
displays it nicely on the side pannel view:

<img src="https://github-public-assets.s3.us-west-1.amazonaws.com/AI_medical_scribe_BertScore_v1_missing_info.png" alt="AI Medical Scribe BertScore v2" width="90%" />

<style>
    @media (prefers-color-scheme: dark) {
        .adaptive-text { background-color: #fc4949; } /* Light text in dark theme */
    }
    @media (prefers-color-scheme: light) {
        .adaptive-text { background-color: #fcbdbd; } /* Dark text in light theme */
    }
</style>

⚠️⚠️⚠️ **Looks like some information was lost during AI-generated summarization** ⚠️⚠️⚠️

The phrase <span class="adaptive-text">We'll also keep you here for a bit for 
observation. We're planning to do an X-ray and an ECG to check your heart and lungs more 
closely. This will help us understand what's causing your chest pain.</span> is pretty 
critical, but isn't in the AI-generated summary.

**MyMedScribe** uses the insights coming from [Lynxius](https://www.lynxius.ai/) to further 
debug and improve their product. After a few days of working **MyMedScribe** team realizes 
that there was a problem with the way they were chunking the script sentences. After 
fixing the issue, they are ready to test again:

In [4]:
# load the new AI-generated summary
new_ai_summary = load_input('./data/AI_medical_scribe_new_summary.yaml')

In [5]:
# Calculate the BertScore and upload results to Lynxius Platform
client.evaluate(
    title="Bert Score v2",
    evaluator="bert_score",
    reference=raw_transcript,
    output=new_ai_summary,
    level="sentence",
    presence_threshold=0.65,
)

Success: {"uuid":"c8db892c-991c-4bd4-930c-c64280bdc851"}


Let's remember the values of the Bert Score Thresholds set for this project:

<table>
  <thead style="background-color: #919191;">
    <tr>
      <th>Metric</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Precision</td>
      <td>0.75</td>
    </tr>
    <tr>
      <td>Recall</td>
      <td>0.55</td>
    </tr>
    <tr>
      <td>F1</td>
      <td>0.60</td>
    </tr>
  </tbody>
</table>


Below a screenshot of the new Bert Score results from [Lynxius Platform](platform.lynxius.ai).

<img src="https://github-public-assets.s3.us-west-1.amazonaws.com/AI_medical_scribe_BertScore_v2.png" alt="AI Medical Scribe BertScore v2" width="90%" />

🎉🎉🎉 **Hurray!!!** 🎉🎉🎉

The AI-generated summary is finally accurate ✅✅✅. **MyMedScribe** team now wants 
to evaluate now their App's performance with filing a clinical documentation report.

## Task-2: Evaluating Writing Quality

**MyMedScribe** App uses the new AI-generated summary to generate the clinical 
documentation report.

<img src="https://github-public-assets.s3.us-west-1.amazonaws.com/AI_medical_scribe_clinical_notes_with_error.png" alt="AI Medical Scribe clinical notes with error" width="60%" />

⚠️⚠️⚠️ **The AI-generated clinical documentation looks good, but... How good is it?** ⚠️⚠️⚠️

**MyMedScribe** team knows that Dr. Taylor has to file the clinical documentation 
electronically and share the report with other doctors. The helath of Brenda, as well as 
Dr. Taylor's reputation are at stake. The report must respect all the typical writing 
quality guidelines and must contain all relevant information at the right place. 
**MyMedScribe** team must evaluate their product before and after deploying it to 
production and chooses [Lynxius](https://www.lynxius.ai/) for testing and evaluating 
their LLM App.

### Contains Medication Prescription Evaluator

**MyMedScribe** team wants to make sure that AI-generated  the clinical documentation 
report always contains a drug prescription and that this is always written in a specific 
order: medication, dosage, frequency.

[Lynxius](https://www.lynxius.ai/) provides an easy interface to build use-case specific 
**custom evaluators**. Custom evaluators can be:
* Heuristics
* AI-assisted

In our case, **MyMedScribe** team needs an AI-assisted evaluator. To implement one, they 
only need to create a new prompt that specifies the use-case dependent policy to respect.

<img src="https://github-public-assets.s3.us-west-1.amazonaws.com/AI_medical_scribe_eval_clinical_notes.png" alt="AI Medical Scribe eval clinical notes" width="60%" />

Let's build the `MedicationPrescription` evaluator:

In [6]:
# Building the evaluator prompt template
MEDICATION_PRESCRIPTION_BASE_TEMPLATE = """
You are given a clinical documentation report as output text and a list of 
medications. You must determine whether the given text report contains one medication 
prescription. The medication prescription must always be written in this order: 
medication, dosage and frequency. The medication must belong to the list of medications 
received as input list. Here is the data:
    [BEGIN DATA]
    ************
    [Output]: {output}
    ************
    [List]: {list}
    [END DATA]
Your response must be composed of a score and an an explanation. Here is the output 
format:
    [BEGIN OUTPUT]
    **************
    [Score]: score here
    **************
    [Explanation]: explanation here
    [END OUTPUT]
Your score must be a single word, either "correct" or "incorrect", and should not 
contain any text or characters aside from that word.
"correct" means that the output text contains one medication prescription and the 
medication prescription is written in the order: medication, dosage and frequency, where 
the medication belongs to the input list of medications.
"incorrect" means that the output text doesn not contains a medication prescription, 
or this is written in a different order, or the medication does not belong to the input 
list.
Your explanation must be a short, step by step description of why you asigned this score 
to be sure your conclusion is correct. Avoid simply stating the correct answers at the 
outset.
"""

Now that we created the `MedicationPrescription` evaluator, let's use it to assess the 
writing quality of **MyMedScribe** App:

In [7]:
# open AI-generated clinical documentation report and generate reference
ai_clinical_docs = load_yaml_input('./data/AI_medical_scribe_clinical_notes.yaml')

In [8]:
# Calculate the BertScore and upload results to Lynxius Platform
client.evaluate(
    title="MedicationPrescription v1",
    evaluator="custom_eval",
    output=ai_clinical_docs,
    str_list="[Atorvastatin, Metformin, Amoxicillin, Novasc, Albuterol]",
    prompt_template=MEDICATION_PRESCRIPTION_BASE_TEMPLATE,
)

Success: {"uuid":"294ef63c-848b-457e-842c-eadaad510b8f"}


Below a screenshot of the MedicationPrescription results from [Lynxius Platform](platform.lynxius.ai).

<img src="https://github-public-assets.s3.us-west-1.amazonaws.com/AI_medical_scribe_MedicationPrescription_v1.png" alt="AI Medical Scribe MedicationPrescription v1" width="90%" />

❌❌❌ **Ouch!!! something went wrong** ❌❌❌

It seems like the `MedicationPrescription` is **INCORRECT**. Thankfully 
[Lynxius](https://www.lynxius.ai/) offers an explenation of the error. It seems like the 
prescription does not include the medication's frequency of use.

<img src="https://github-public-assets.s3.us-west-1.amazonaws.com/AI_medical_scribe_clinical_notes_comparison.png" alt="AI Medical Scribe clinical notes comparison" width="60%" />

**MyMedScribe** team uses this insights to improve their product and after a few days of 
working, they fixed the issue and are ready to test again with the newly AI-generated 
clinical documentation report:

In [9]:
# open new AI-generated clinical documentation report and generate reference
new_ai_clinical_docs = load_yaml_input('./data/AI_medical_scribe_new_clinical_notes.yaml')

In [10]:
# Calculate the BertScore and upload results to Lynxius Platform
client.evaluate(
    title="MedicationPrescription v2",
    evaluator="custom_eval",
    output=new_ai_clinical_docs,
    str_list="[Atorvastatin, Metformin, Amoxicillin, Novasc, Albuterol]",
    prompt_template=MEDICATION_PRESCRIPTION_BASE_TEMPLATE,
)

Success: {"uuid":"8787e806-e20c-4848-8619-700478a26a95"}


Below a screenshot of the new MedicationPrescription results from [Lynxius Platform](platform.lynxius.ai).

<img src="https://github-public-assets.s3.us-west-1.amazonaws.com/AI_medical_scribe_MedicationPrescription_v2.png" alt="AI Medical Scribe MedicationPrescription v2" width="90%" />

🎉🎉🎉 Hurray!!! 🎉🎉🎉

The clinical documentation report is finally accurate ✅✅✅. **MyMedScribe** team can 
finally deploy the newest version of their LLM App to production 🛠️🛠️🛠️ They can also 
keep using `BERTScore` and `MedicationPrescription` and many more 
[Lynxius](https://www.lynxius.ai/) evaluators to monitor the production behaviour of 
their product.